<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_p25/blob/main/professor/numpy/notebooks/07_Vectorizacion_vs_For_vs_Comprehensions.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 07 — Vectorización vs for vs comprensiones

Este cuaderno explica con claridad, en español, cuándo conviene usar bucles de Python, comprensiones de listas u operaciones vectorizadas con NumPy. Nos enfocamos en rendimiento, legibilidad y escalabilidad.

- Bucles `for` en Python
- Comprensiones de listas
- Operaciones vectorizadas con NumPy (ufuncs y broadcasting)

También aprenderás a medir correctamente con `timeit`, qué sesgos evitar y qué reglas prácticas usar para decidir.

## ¿Qué es “vectorizar”?
Vectorizar es expresar una operación sobre un conjunto completo de datos (arreglos, matrices) sin escribir bucles explícitos. NumPy implementa funciones universales (ufuncs) en C y soporta broadcasting; así reduce el trabajo del intérprete y aprovecha mejor la memoria y el hardware (SIMD).

## ¿Por qué un `for` suele ser más lento?
- Cada iteración en Python tiene un coste fijo del intérprete (gestión de objetos, despacho de bytecode).
- La aritmética con `int`/`float` de Python opera con objetos; en NumPy se opera sobre buffers contiguos y tipos fijos (`float32`/`float64`).

## Comprensiones de listas
- Suelen ser más rápidas que un `for` manual porque parte del trabajo ocurre en C, pero siguen pagando coste por elemento y no aprovechan SIMD/BLAS como NumPy.

## NumPy: ufuncs y broadcasting
- Las ufuncs aplican la operación elemento a elemento en C sobre memoria contigua.
- El broadcasting combina arreglos de formas compatibles (p. ej. `(n, 1)` con `(1, m)`) sin copiar datos.

## Cómo medir con `timeit`
- Ejecuta varias repeticiones y reporta la mediana.
- Compara trabajos equivalentes (materializa generadores si hace falta).
- Evita E/S y cualquier código no esencial dentro de la sección a medir.

## Qué vamos a comparar
- Multiplicación simple: `x * 2`
- Funciones trigonométricas: `sin`, `cos`
- Generadores: pereza (lazy) vs materialización
- Preasignación vs `append` en Python y estrategias en NumPy
- `np.vectorize`: qué hace y por qué no acelera
- Dtypes y memoria: `float32` vs `float64`


## Metodología de benchmarking y buenas prácticas

- Repeticiones y mediana: realiza varias repeticiones y reporta la mediana para amortiguar outliers.
- Trabajo comparable: mide exactamente el mismo trabajo; si usas un generador, materialízalo para poder compararlo con NumPy.
- Calentamiento implícito (warm‑up): repetir llamadas calienta cachés e importa módulos; la mediana ayuda a estabilizar los tiempos.
- Tamaño del problema: elige un `n` que haga visible el coste por iteración sin agotar memoria.
- Verificación de resultados: valida primero que las implementaciones produzcan el mismo resultado (con tolerancia numérica si procede).
- Evita E/S: no imprimas ni escribas a disco dentro del código bajo prueba.



### Nota: ¿Qué es `np.allclose`?

`np.allclose(a, b, rtol=1e-05, atol=1e-08, equal_nan=False)` comprueba si dos arreglos son “casi iguales” elemento a elemento, dentro de tolerancias numéricas.

- `rtol` (tolerancia relativa): permite pequeñas diferencias proporcionales al valor.
- `atol` (tolerancia absoluta): margen fijo permitido, útil cerca de 0.
- Devuelve `True` si para todos los elementos se cumple `|a - b| <= atol + rtol * |b|`.

Es útil con números de punto flotante, donde los redondeos hacen que comparaciones exactas fallen. Si necesitas igualdad exacta, usa `np.array_equal`.



In [5]:
import numpy as np
import math
import timeit
from statistics import median

# Datos base para los experimentos
n = 200_000
arr = np.arange(n, dtype=np.float64)

# Implementaciones equivalentes (multiplicar por 2)
def times_two_for(a):
    out = []
    append = out.append  # micro-optimización local
    for x in a:
        append(x * 2)
    return out

def times_two_comp(a):
    return [x * 2 for x in a]

def times_two_np(a):
    return a * 2

# Utilidades de benchmarking

def measure(stmt, number=10, repeat=7):
    """Mide un callable sin argumentos y devuelve la mediana del tiempo por llamada.

    Parámetros:
    - stmt: callable sin argumentos a medir.
    - number: número de ejecuciones por repetición (se promedian dentro de cada repetición).
    - repeat: número de repeticiones independientes; se toma la mediana entre repeticiones.

    Devuelve:
    - float: segundos por llamada = mediana(repeticiones) / number.

    Notas:
    - Usa timeit.repeat para reducir ruido y outliers.
    - Si necesitas pasar argumentos, envuélvelos con una lambda o functools.partial.
    """
    times = timeit.repeat(stmt, number=number, repeat=repeat)
    return median(times) / number

def ns_per_element(seconds_per_call, elements):
    return seconds_per_call / elements * 1e9

def compare(label_to_callables, elements=n):
    """Recibe un dict {label: callable} y devuelve dict con métricas por label."""
    results = {}
    for label, fn in label_to_callables.items():
        spc = measure(fn)
        results[label] = {
            "s/call": spc,
            "ns/elem": ns_per_element(spc, elements),
        }
    return results

# Chequeo de correctitud (sanity check)
small = np.arange(10, dtype=np.float64)
assert np.allclose(np.array(times_two_for(small)), small * 2)
assert np.allclose(np.array(times_two_comp(small)), small * 2)
assert np.allclose(times_two_np(small), small * 2)

print("Utilidades listas. Ejecuta las celdas de comparación más abajo.")


Utilidades listas. Ejecuta las celdas de comparación más abajo.


In [6]:
# Generadores (lazy) vs NumPy y materialización justa

def times_two_gen(a):
    return (x * 2 for x in a)  # lazy: no hace trabajo hasta consumir

# Medimos tres cosas:
# 1) Crear el generador (barato)
# 2) Materializar a lista 
# 3) NumPy vectorizado 

create_gen_s = measure(lambda: times_two_gen(arr))
materialize_gen_s = measure(lambda: list(times_two_gen(arr)))
vectorized_s = measure(lambda: times_two_np(arr))

print({
    "crear_generador s/call": create_gen_s,
    "materializar_generador s/call": materialize_gen_s,
    "numpy s/call": vectorized_s,
})


{'crear_generador s/call': 8.256000000983477e-07, 'materializar_generador s/call': 0.031121767600006935, 'numpy s/call': 9.723800001211202e-05}


## Experimentos base: `for` vs comprensiones vs NumPy

Comparamos implementaciones equivalentes para multiplicar por 2. Reportamos tiempo por llamada y nanosegundos por elemento.

Notas:
- Las listas de Python almacenan objetos; NumPy opera con tipos fijos en memoria contigua.
- Resultado esperable: `NumPy` >> `comprensión` > `for`.



In [7]:
base_results = compare({
    "for": lambda: times_two_for(arr),
    "comp": lambda: times_two_comp(arr),
    "numpy": lambda: times_two_np(arr),
})

# Presentación simple
for k, v in base_results.items():
    print(k, {kk: round(vv, 3) for kk, vv in v.items()})


for {'s/call': 0.028, 'ns/elem': 138.908}
comp {'s/call': 0.025, 'ns/elem': 127.135}
numpy {'s/call': 0.0, 'ns/elem': 0.387}


## Funciones trigonométricas: `math.sin` vs `np.sin`

Las ufuncs (`np.sin`, `np.cos`) están vectorizadas y escritas en C. En Python puro, `math.sin` se invoca por elemento dentro de un bucle o de una comprensión.

- Aquí la diferencia suele ser mayor que en aritmética simple, porque cada llamada a `math.sin` añade coste por elemento.



In [8]:
def sin_for(a):
    out = []
    append = out.append
    for x in a:
        append(math.sin(x))
    return out

def sin_comp(a):
    return [math.sin(x) for x in a]

def sin_np(a):
    return np.sin(a)

# Correctitud
small = np.linspace(0, 1, 10)
assert np.allclose(np.array(sin_for(small)), np.sin(small))
assert np.allclose(np.array(sin_comp(small)), np.sin(small))

trig_results = compare({
    "for_sin": lambda: sin_for(arr),
    "comp_sin": lambda: sin_comp(arr),
    "numpy_sin": lambda: sin_np(arr),
}, elements=n)

for k, v in trig_results.items():
    print(k, {kk: round(vv, 3) for kk, vv in v.items()})


for_sin {'s/call': 0.039, 'ns/elem': 194.074}
comp_sin {'s/call': 0.043, 'ns/elem': 216.396}
numpy_sin {'s/call': 0.003, 'ns/elem': 13.585}


## Cómo vectorizar: guía rápida con ejemplos

La idea principal es reemplazar bucles explícitos por operaciones sobre arreglos completos.

- Usa ufuncs: `np.add`, `np.multiply`, `np.sin`, `np.exp`, etc.
- Aprovecha broadcasting para combinar arreglos de formas compatibles sin copiar datos.
- Evita `for` cuando la operación sea elemento a elemento y exista ufunc equivalente.



In [9]:
# Ejemplo 1: suma y escala sin bucles
x = np.arange(10, dtype=np.float64)
# bucle (evitar)
res_for = [xi * 2 + 3 for xi in x]
# vectorizado (preferir)
res_np = x * 2 + 3
assert np.allclose(np.array(res_for), res_np)
res_np


array([ 3.,  5.,  7.,  9., 11., 13., 15., 17., 19., 21.])

### Broadcasting: combinar formas distintas

Regla mental:
- Dimensiones iguales o 1 son compatibles.
- NumPy “estira” (sin copiar) la dimensión 1 para que coincida.



In [10]:
# Ejemplo 2: distancia euclidiana por filas con broadcasting
# Queremos distancias entre puntos de A (n,2) y un centro c (2,)
A = np.array([[0., 0.], [1., 1.], [2., 2.]])
c = np.array([1., 0.])

# Forma con bucle (evitar)
res_loop = [np.sqrt((p[0]-c[0])**2 + (p[1]-c[1])**2) for p in A]

# Forma vectorizada (preferir)
# A - c usa broadcasting: (n,2) - (2,) -> (n,2)
res_vec = np.sqrt(((A - c) ** 2).sum(axis=1))

assert np.allclose(np.array(res_loop), res_vec)
res_vec


array([1.        , 1.        , 2.23606798])

## Preasignación vs `append`

En Python puro, `append` dentro de un bucle añade coste por operación. Cuando conozcas el tamaño final, es mejor preasignar y rellenar por índice. En NumPy, evita crecer arreglos en un bucle; crea el arreglo final de una vez y aplica operaciones vectorizadas.



In [11]:
m = 200_000

# Python: append vs preasignación

def squares_append_py(m):
    out = []
    append = out.append
    for i in range(m):
        append(i * i)
    return out

def squares_prealloc_py(m):
    out = [0] * m
    for i in range(m):
        out[i] = i * i
    return out

# NumPy: crear y rellenar de golpe

def squares_numpy(m):
    a = np.arange(m, dtype=np.int64)
    return a * a

# Correctitud
assert squares_append_py(5) == squares_prealloc_py(5) == list((np.arange(5) ** 2))

prealloc_results = {
    "append_py s/call": measure(lambda: squares_append_py(m)),
    "prealloc_py s/call": measure(lambda: squares_prealloc_py(m)),
    "numpy s/call": measure(lambda: squares_numpy(m)),
}

for k, v in prealloc_results.items():
    print(k, round(v, 6))


append_py s/call 0.015455
prealloc_py s/call 0.015958
numpy s/call 0.001853


## `np.vectorize`: aclaración importante

`np.vectorize` NO acelera por sí mismo: es un envoltorio que llama a tu función Python por elemento. Sirve para escribir código más compacto, no para ganar rendimiento. Si existe una ufunc nativa, úsala.



In [12]:
def slow_py_func(x):
    # una función arbitraria en Python
    return math.sin(x) + x * x

vec_slow = np.vectorize(slow_py_func)

# Comparamos llamadas
vectorize_results = {
    "for_py s/call": measure(lambda: [slow_py_func(x) for x in arr]),
    "vectorize s/call": measure(lambda: vec_slow(arr)),
    "numpy_ufunc s/call": measure(lambda: np.sin(arr) + arr * arr),
}

for k, v in vectorize_results.items():
    print(k, round(v, 6))


for_py s/call 0.071427
vectorize s/call 0.061673
numpy_ufunc s/call 0.004767


## Dtypes y memoria: `float32` vs `float64`

- `float64` ocupa el doble de memoria que `float32` y puede ser más lento en operaciones con grandes volúmenes de datos por presión de memoria/caché.
- Elige el dtype mínimo que preserve la precisión que necesitas.



In [13]:
# Comparamos memoria y tiempos a gran escala
n_big = 5_000_000
x32 = np.arange(n_big, dtype=np.float32)
x64 = np.arange(n_big, dtype=np.float64)

print({
    "x32_MB": round(x32.nbytes / 1e6, 1),
    "x64_MB": round(x64.nbytes / 1e6, 1),
})

mem_results = {
    "sum32 s/call": measure(lambda: np.sum(x32)),
    "sum64 s/call": measure(lambda: np.sum(x64)),
}

for k, v in mem_results.items():
    print(k, round(v, 6))


{'x32_MB': 20.0, 'x64_MB': 40.0}
sum32 s/call 0.00175
sum64 s/call 0.002738


## Patrones y anti‑patrones al vectorizar

Usa estos patrones para escribir código claro y rápido con NumPy, y evita los anti‑patrones comunes que degradan el rendimiento o la claridad.

### Patrones (haz esto)
- Usa ufuncs y expresiones vectorizadas (suma, multiplicación, trigonometría).
- Aplica broadcasting para combinar formas compatibles sin copias.
- Emplea agregaciones de NumPy (`np.sum`, `np.mean`, `np.max`, ...).
- Filtra y transforma con máscaras booleanas y `np.where`.
- Preasigna o calcula de una vez; evita crecer arreglos en bucles.
- Elige `dtype` mínimo suficiente; usa operaciones in‑place o `out=` cuando convenga.

### Anti‑patrones (evita esto)
- Iterar en Python sobre `ndarray` para operaciones elemento a elemento.
- Usar `np.vectorize` para “acelerar” (no acelera).
- Hacer `np.append`/`np.concatenate` dentro de bucles.
- Convertir ida y vuelta entre listas y `ndarray` en cada iteración.
- Usar `sum`/`max`/`min` de Python sobre arreglos en lugar de las funciones de NumPy.
- Crear copias innecesarias; abusar de `.copy()` sin necesidad.



In [14]:
# Anti‑patrón 1: bucle Python para suma y escala
x = np.arange(8, dtype=np.float64)
mal = [xi * 2 + 3 for xi in x]
# Patrón correcto: expresión vectorizada
bien = x * 2 + 3
assert np.allclose(np.array(mal), bien)
bien


array([ 3.,  5.,  7.,  9., 11., 13., 15., 17.])

In [15]:
# Anti‑patrón 2: concatenar en bucle
parts = [np.ones((1000,), dtype=np.float64) for _ in range(10)]
# Evitar esto: crece en cada paso (O(n^2) tiempo)
mal = np.array([], dtype=np.float64)
for p in parts:
    mal = np.concatenate([mal, p])

# Patrón correcto: concatenar una vez
bien = np.concatenate(parts)
assert np.allclose(mal, bien)
len(bien)


10000

In [16]:
# Anti‑patrón 3: vectorize para acelerar
# Incorrecto: np.vectorize sigue llamando a Python por elemento
f = lambda t: math.sin(t) + t
vf = np.vectorize(f)
mal = vf(x)
# Correcto: combinar ufuncs nativas
bien = np.sin(x) + x
assert np.allclose(mal, bien)



In [17]:
# Patrón: máscaras booleanas y np.where
x = np.arange(-4, 5)
mask = x < 0
# Reemplazar negativos por 0
bien = np.where(mask, 0, x)
# Equivalente con bucle (evitar)
mal = [0 if xi < 0 else xi for xi in x]
assert np.allclose(np.array(mal), bien)
bien


array([0, 0, 0, 0, 0, 1, 2, 3, 4])

In [18]:
# Mini ejemplo: np.array_equal vs np.allclose
# Igualdad exacta falla por redondeo; allclose pasa con tolerancias
x = np.array([0.1 + 0.2])
y = np.array([0.3])
print("array_equal:", np.array_equal(x, y))
print("allclose:", np.allclose(x, y))


array_equal: False
allclose: True


## Conclusiones prácticas

- Vectoriza siempre que haya una ufunc que cubra tu operación.
- Usa broadcasting para evitar bucles anidados y copias innecesarias.
- Prefiere `float32` cuando la precisión lo permita y el volumen de datos sea grande.
- Evita `np.vectorize` si tu objetivo es acelerar; es solo azúcar sintáctico.
- Mide con `timeit` usando mediana y trabajo comparable.

## Ejercicios sugeridos

1) Implementa `y = 3*x**2 + 2*x + 1` con `for`, comprensión y NumPy, y compara tiempos.
2) Calcula distancias de cada punto de `A (n,2)` a cada punto de `B (m,2)` con broadcasting.
3) Repite las pruebas con `float32` y `float64` y compara memoria/tiempos.
4) Escribe una función Python cualquiera y contrástala con una combinación de ufuncs de NumPy.

